In [1]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,3):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.pcbox.com/ordenadores/ordenadores-portatiles?page={i}")
    
    articles = r.html.find('article')
    links = [article.find('a', first=True).attrs['href'] for article in articles]
    portatil_links.extend(links)

Pagina 1


AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
porta